In [1]:
import pandas as pd
from housing_elements import utils

In [19]:
# Oakland and Palo Alto don't work right now (parsing certain columns as floats fails)
cities = ['San Francisco', 'Berkeley', 'Mountain View', 'San Jose']

In [23]:
def get_results_for_city(city: str) -> pd.DataFrame:
    permits = utils.load_all_new_building_permits(city)
    sites = utils.load_site_inventory(city)
    
    return {
        'City': city,
        'Ratio of development on inventory sites': utils.calculate_total_units_permitted_over_he_capacity(sites, permits),
        'Mean overproduction': utils.calculate_mean_overproduction_on_sites(sites, permits),
        'Inventory housing / all housing': utils.calculate_inventory_housing_over_all_housing(sites, permits),
        'P(dev) for nonvacant sites': utils.calculate_pdev_for_nonvacant_sites(sites, permits),
        'P(dev) for vacant sites': utils.calculate_pdev_for_vacant_sites(sites, permits),
        'P(dev) for inventory': utils.calculate_pdev_for_inventory(sites, permits),
    }

In [24]:
%%time
results_df = pd.DataFrame([get_results_for_city(city) for city in cities])

/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (6158, 16)
Total units permitted: 26071
Total realistic capacity in inventory: 47209.0
Number of inventory sites developed: 0
Number of units permitted on inventory sites: 0
Total realistic capacity of inventory sites: 0.0
Units permitted on inventory sites: 0
Total units permitted: 26071


/Users/sidharthkapur/personal-workspace/housing-elements/housing_elements/utils.py:299: RuntimeWarning: invalid value encountered in double_scalars
  return (n_units - n_claimed) / len(inventory_sites_permitted)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (404, 18)
Total units permitted: 1871
Total realistic capacity in inventory: 5372.02
Number of inventory sites developed: 0
Number of units permitted on inventory sites: 0
Total realistic capacity of inventory sites: 0.0
Units permitted on inventory sites: 0
Total units permitted: 1871


/Users/sidharthkapur/personal-workspace/housing-elements/housing_elements/utils.py:299: RuntimeWarning: invalid value encountered in double_scalars
  return (n_units - n_claimed) / len(inventory_sites_permitted)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'num_vl' 'num_l' 'num_m'
 'num_am']
DF shape (93, 24)
Total units permitted: 3579.0
Total realistic capacity in inventory: 3216.0
Number of inventory sites developed: 3
Number of units permitted on inventory sites: 426.0
Total realistic capacity of inventory sites: 799.0
Units permitted on inventory sites: 426.0
Total units permitted: 3579.0


/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (599, 18)
Total units permitted: 16928.0
Total realistic capacity in inventory: 35536.43
Number of inventory sites developed: 64
Number of units permitted on inventory sites: 3473.0
Total realistic capacity of inventory sites: 3664.0
Units permitted on inventory sites: 3473.0
Total units permitted: 16928.0
CPU times: user 45 s, sys: 1.18 s, total: 46.2 s
Wall time: 47 s


In [25]:
results_df

,City,Ratio of development on inventory sites,Mean overproduction,Inventory housing / all housing,P(dev) for nonvacant sites,P(dev) for vacant sites,P(dev) for inventory
0,San Francisco,0.552246,NaN,0.000000,0.000000,0.000000,0.000000
1,Berkeley,0.348286,NaN,0.000000,0.000000,0.000000,0.000000
2,Mountain View,1.112873,-124.333333,0.119028,0.023256,0.142857,0.032258
3,San Jose,0.476356,-2.984375,0.205163,0.127226,0.067961,0.106845
